# DATA 608 Assignment 5: What Is The Effect Of The Earth's Temperature on Cyclonic Storms?

Historic data indicates that the occurrence and intensity of cyclonic storms (Hurricanes, Typhoons and Tornados) increases with the increased earth temperature. 

For this assignment you will need to tell this story to a non-technical audience (eg: a high-school earth science class)



In [69]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

# Data Preparation

In [71]:
# load the data set for earth anomalies
anomalies_url = 'https://raw.githubusercontent.com/eddiexunyc/earth_temp_cyclone_data/refs/heads/main/Resources/anomalies_data.csv'
anomalies_data = pd.read_csv(anomalies_url)

# load the hurricane data set and remove uncessary columns
hur_data_url = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2023-051124.txt'
hurricane_raw_data = pd.read_csv(hur_data_url, sep = ',').reset_index()
hurricane_raw_data = hurricane_raw_data.drop(['AL011851', '            UNNAMED', '     14', 'Unnamed: 3','level_7',
                                               'level_8', 'level_9', 'level_10', 'level_11', 'level_12', 
                                              'level_13', 'level_14', 'level_15', 'level_16'], axis=1)

The hurricane data is sourced from NOAA Hurricane Database and is unstructured. Because of that, it needs to be transformed before loading it for data analysis and visualization.

In [74]:
# create a dictionary and sort the data based on the storm ID and name
hurricane_raw_data_dict = hurricane_raw_data.to_dict(orient='record')
storm_id = None
storm_name = None
date_time = None
latitude = None
longtitude = None
wind_speed = None

hurricane_data = []

for data in hurricane_raw_data_dict:
    if data['level_0'].startswith('AL'):
        storm_id = data['level_0']
        storm_name = data['level_1']
    else:
        date_time = data['level_0']
        latitude = data['level_4']
        longtitude = data['level_5']
        wind_speed = data['level_6']

    hurricane_data.append({
        'storm_id': storm_id,
        'storm_name': storm_name,
        'date_time': date_time,
        'latitude': latitude,
        'longtitude': longtitude,
        'wind_speed': wind_speed
    })

# convert the dictionary back to a data frame
hurricane_data = pd.DataFrame(hurricane_data)
hurricane_data['date_time'] = pd.to_datetime(hurricane_data['date_time'], format='%Y-%m-%d')
hurricane_data['year'] = hurricane_data['date_time'].dt.year

# extract the data starting from 1999 and group by
hurricane_group_data = hurricane_data[hurricane_data['year'] >= 1999]
hurricane_group_data = round(hurricane_group_data.groupby(['storm_name', 'year'])['wind_speed'].mean(), 2).reset_index().sort_values(['year'])
hurricane_year_data = round(hurricane_group_data.groupby(['year'])['wind_speed'].mean(), 2).reset_index().sort_values(['year'])

# combine both hurricane and earth temperature anomlias data
cyclone_year_data = hurricane_year_data.merge(anomalies_data, how = 'left', left_on = 'year', right_on = 'Year')
cyclone_year_data = cyclone_year_data.drop(['Year'], axis=1)

cyclone_year_data

/var/folders/h4/zjq554hs0b57vqfcrc5738wh0000gn/T/ipykernel_12961/3237840897.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  hurricane_raw_data_dict = hurricane_raw_data.to_dict(orient='record')


,year,wind_speed,Anomaly
0,1999,53.62,0.44
1,2000,45.66,0.43
2,2001,45.36,0.48
3,2002,37.20,0.64
4,2003,44.67,0.60
5,2004,47.53,0.58
6,2005,44.95,0.66
7,2006,43.66,0.66
8,2007,39.56,0.69
9,2008,43.84,0.52


# Data Visualization

In [ ]:
# line graph
fig, ax1 = plt.subplots(figsize=(12, 8))
ax1 = 